# Descarga de series temporales del modelo MGB a partir de la API de SSTD-CIC
Página de test: https://sstdfews.cicplata.org/FewsWebServices/index.html 

In [1]:
import requests
from datetime import datetime, timezone
import pandas
from plotly import express as px

def parseDate(date : dict):
    """parse date-time dict

    Args:
        date (dict): {"date" : str, "time" : str}

    Returns:
        datetime: datetime in utc timezone
    """
    d = [int(x) for x in date["date"].split("-")]
    t = [int(x) for x in date["time"].split(":")]
    return datetime(d[0],d[1],d[2],t[0],t[1],t[2], tzinfo=timezone.utc)

# Request data using GET /timeseries
    
    curl -X GET "https://sstdfews.cicplata.org/FewsWebServices/rest/fewspiservice/v1/timeseries?locationIds=6362&parameterIds=Q.sim&moduleInstanceIds=MGB_Forecast&startTime=2024-01-01T00%3A00%3A00Z&endTime=2025-03-31T00%3A00%3A00Z&documentFormat=PI_JSON" \
    -H "Accept: application/json" \



In [2]:

location_id = 6362
parameter_id = "Q.sim"
module_instance_id = "MGB_Forecast"
start_time = "2024-01-01T00:00:00Z"
end_time = "2025-03-20T00:00:00Z"

response = requests.get(
  "https://sstdfews.cicplata.org/FewsWebServices/rest/fewspiservice/v1/timeseries",
  params= {
    "locationIds": location_id,
    "parameterIds": parameter_id,
    "moduleInstanceIds": module_instance_id,
    "startTime": start_time,
    "endTime": end_time,
    "documentFormat": "PI_JSON"
  })

timeseries = response.json()


In [ ]:
# len(timeseries["timeSeries"])
# timeseries["timeSeries"][0]["header"]
# len(timeseries["timeSeries"][0]["events"])

## extract valuable metadata and time-value pairs

In [6]:
def parseTimeSeries(ts : dict):
    variable = {
        "id": ts["header"]["parameterId"],
        "dt": {
        "seconds": int(ts["header"]["timeStep"]["multiplier"])
        },
        "units": ts["header"]["units"]
    }
    station = {
        "id": ts["header"]["locationId"],
        "name": ts["header"]["stationName"],
        "geometry": {
        "type": "Point",
        "coordinates": [
            float(ts["header"]["lon"]),
            float(ts["header"]["lat"])
        ]
        }
    }
    forecast = {
        "date": parseDate(ts["header"]["forecastDate"]),
        "values": [
            {
                "date": parseDate({"date": x["date"], "time": x["time"]}),
                "value": float(x["value"])
            } 
            for x in ts["events"] if float(x["value"]) != -999
        ]
    }
    return {
        "variable": variable,
        "station": station,
        "forecast": forecast
    }


In [7]:
ts = parseTimeSeries(timeseries["timeSeries"][0])

## forecast time-value pairs to DataFrame

In [9]:
forecast_df = pandas.DataFrame([
    {
        "date": v["date"],
        "value": v["value"]
    }
    for v in ts["forecast"]["values"]
])

## plot

In [10]:
title = "%s - %s [%s] - %s [%s]" % (module_instance_id, ts["variable"]["id"], ts["variable"]["units"], ts["station"]["name"], ts["station"]["id"]) # 'MGB Q.sim - Rosario'

fig = px.line(forecast_df.head(-1), x="date", y="value", title=title)
fig.show()

## request data using /timeseries/displaygroups (all locations and parameters)

In [11]:
start_time = "2024-01-01T00:00:00Z"
end_time = "2025-03-20T00:00:00Z"

response = requests.get(
  "https://sstdfews.cicplata.org/FewsWebServices/rest/fewspiservice/v1/timeseries/displaygroups",
  params= {
    "plotId": "MGB",
    "startTime": start_time,
    "endTime": end_time,
    "documentFormat": "PI_JSON"
  })

display_groups = response.json()

## extract valuable metadata and time-value pairs

In [12]:
series = []
for ts in display_groups["timeSeries"]:
  series.append(parseTimeSeries(ts))
  

## series metadata to dataframe

In [13]:
series_df = pandas.DataFrame(
    [
        {
            "parameterId": ts["header"]["parameterId"],
            "units": ts["header"]["units"],
            "timeStep_unit": ts["header"]["timeStep"]["unit"],
            "timeStep_multiplier": int(ts["header"]["timeStep"]["multiplier"]),
            "startDate": parseDate(ts["header"]["startDate"]),
            "endDate": parseDate(ts["header"]["endDate"]),
            "locationId": ts["header"]["locationId"],
            "stationName": ts["header"]["stationName"],
            "lon": float(ts["header"]["lon"]),
            "lat": float(ts["header"]["lat"]),
            "forecastDate": parseDate(ts["header"]["forecastDate"])
        }
        for ts in display_groups["timeSeries"]
    ]
)

## find metadata using parameterId and locationId. Convert to dict

In [14]:
location_id = "6362"
parameter_id = "Q.sim"

serie = series_df.loc[series_df.locationId==location_id].loc[series_df.parameterId==parameter_id].reset_index().to_dict(orient='records')[0]

## forecast to dataframe

In [15]:
forecast_df = pandas.DataFrame([
    {
        "date": v["date"],
        "value": v["value"]
    }
    for v in series[serie["index"]]["forecast"]["values"]
])

## plot

In [16]:
title = 'MGB Q.sim - Rosario'

fig = px.line(forecast_df.head(-1), x="date", y="value", title=title)
fig.show()